In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("train.csv")

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test = pd.read_csv("test.csv")

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
sample_submission = pd.read_csv("sample_submission.csv")

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.shape

In [ ]:
train.info()

In [ ]:
train.describe()

# Make a copy of the dataset

In [ ]:
train_copy = train.copy()

In [ ]:
train_copy.head()

# Dropping features not needed for the model

In [ ]:
# train = train.drop(columns = 'customer_id')

In [ ]:
# train, test = train.drop(columns = 'job'), test.drop(columns = 'job')

In [ ]:
# train , test = train.drop(columns = 'education'), test.drop(columns = 'education')

In [ ]:
# train, test = train.drop(columns = 'default'), test.drop(columns = 'default')

In [ ]:
train = train.drop(columns=['customer_id', 'poutcome', 'job', 'education', 'default', 'housing', 'loan', 'contact', 'month'])

In [ ]:
print(train.columns)
print( train.shape)

In [ ]:
test = test.drop(columns=['customer_id', 'poutcome', 'job', 'education', 'default', 'housing', 'loan', 'contact', 'month'])

In [ ]:
print(test.columns)
print(test.shape)

In [ ]:
train.info()

# Encoding marital status

In [ ]:
train = pd.get_dummies(train)

In [ ]:
test = pd.get_dummies(test)

In [ ]:
# train.head()

In [ ]:
X = train.drop(columns = 'subscribed')
y= train['subscribed']

# Splitting the dataset into Train and Test

In [ ]:
#Importing split module
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 165)

In [ ]:
# X_train

In [ ]:
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(random_state=42)
# X_res, y_res = sm.fit_resample(X,y)

In [ ]:
# X_res, y_res = sm.fit_resample(X,y)

# Building the Model KNN

In [ ]:
#import knn module
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()
classifier.fit(X, y)

In [ ]:
knn_pred = classifier.predict(test)

In [ ]:
knn_pred.shape

In [ ]:
sub = sample_submission.copy()

In [ ]:
sub.subscribed = knn_pred

In [ ]:
sub.to_csv('5.csv', index= False)

# Build Support Vector Machine Model

In [ ]:
from sklearn.svm import SVC
svm_classifier = SVC(kernel = 'linear')
svm_classifier.fit(X, y)
svm_pred = svm_classifier.predict(test)

In [ ]:
svm_classifier = SVC(kernel = 'linear')

In [ ]:
svm_classifier.fit(X, y)

In [ ]:
svm_pred = svm_classifier.predict(test)

In [ ]:
svm_pred.shape

In [ ]:
sub_svm = sample_submission.copy()

In [ ]:
sub_svm.subscribed = svm_pred

In [ ]:
sub_svm.to_csv('6.csv', index = False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(random_state=42)

In [ ]:
rfc.fit(X, y)
rfc_pred = rfc.predict(test)

In [ ]:
rfc_svm = sample_submission.copy()
rfc_svm.subscribed = rfc_pred
rfc_svm.to_csv('7.csv', index = False)

In [ ]:
#modelling using XGB classifier with thread
import xgboost as xg
xgb = xg.XGBClassifier(nthreads = -1)
xgb.fit(X, y)

In [ ]:
pred_xgb = xgb.predict(test)

In [ ]:
xgb_svm = sample_submission.copy()
xgb_svm.subscribed = pred_xgb
xgb_svm.to_csv('9.csv', index = False)

In [ ]:
#modelling using XGB classifier with thread
import xgboost as xg
xgb = xg.XGBClassifier(n_estimators=1000)
xgb.fit(X, y)
pred_xgb = xgb.predict(test)
xgb_svm = sample_submission.copy()
xgb_svm.subscribed = pred_xgb
xgb_svm.to_csv('13.csv', index = False)

In [ ]:
from catboost import CatBoostClassifier
cat_model=CatBoostClassifier(max_depth=3)
cat_model.fit(X,y)
#when I set the max_depth to 3, my accuracy increased tofrom 0.917 to 0.92

In [ ]:
pred_cat=cat_model.predict(test)
pred_cat2 = cat_model.predict(X_test)
cat_sub = sample_submission.copy()
cat_sub.subscribed = pred_cat
cat_sub.to_csv('16.csv', index = False)

In [ ]:
from sklearn.metrics import f1_score
acc =f1_score(y_test, pred_cat2)

In [ ]:
acc

In [ ]:
CatBoostClassifier?

## FEATURE ENGINEERING

In [ ]:
train2 = pd.read_csv("train.csv")
test2 = pd.read_csv("test.csv")

In [ ]:
# Label encoding for categoricals
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X.dtypes == int

In [ ]:
train2.shape

In [ ]:
from sklearn.feature_selection import mutual_info_classif

In [ ]:
X= train2.drop (columns='subscribed')

In [ ]:
y = train2['subscribed']

In [ ]:
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_classif(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores[::3]  # show a few features with their MI scores

In [ ]:
mi_scores

In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")


plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores)

In [ ]:
X

In [ ]:
test2 =  test2[['duration', 'euribor3m', 'cons_conf_idx', 'nr_employed', 'emp_var_rate', 'pdays', 'poutcome', 'month']]

In [ ]:
test2

In [ ]:
X = pd.get_dummies(X)
test2 = pd.get_dummies(test2)

In [ ]:
X.shape, test2.shape


In [ ]:
# from catboost import CatBoostClassifier
# cat_model=CatBoostClassifier(max_depth=5)
# cat_model.fit(X,y)
# #when I set the max_depth to 3, my accuracy increased tofrom 0.917 to 0.92

In [ ]:
pred_cat=cat_model.predict(test2)

cat_sub = sample_submission.copy()
cat_sub.subscribed = pred_cat
cat_sub.to_csv('18.csv', index = False)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=3)

In [ ]:
dt.fit(X,y)

In [ ]:
dt_pred = dt.predict(test2)
dt_sub = sample_submission.copy()
dt_sub.subscribed = dt_pred
dt_sub.to_csv('19.csv', index = False)

In [ ]:
test3 = pd.read_csv('test.csv')

In [ ]:
test3 =  test3[['duration', 'euribor3m', 'cons_conf_idx', 'nr_employed', 'emp_var_rate', 'pdays', 'poutcome', 'month']]

In [ ]:
test3= pd.get_dummies(test3)

In [ ]:
#using StandardScaler to scale our independent variable
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X= scaler.fit_transform(X)

test3 = scaler.fit_transform(test3)


In [ ]:
# from catboost import CatBoostClassifier
# cat_model=CatBoostClassifier(max_depth=3)
# cat_model.fit(X,y)
# #when I set the max_depth to 3, my accuracy increased tofrom 0.917 to 0.92

In [ ]:
pred_cat=cat_model.predict(test3)
cat_sub = sample_submission.copy()
cat_sub.subscribed = pred_cat
cat_sub.to_csv('21.csv', index = False)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X,y)

knn_pred=knn.predict(test)

knn_sub = sample_submission.copy()
knn_sub.subscribed = knn_pred
knn_sub.to_csv('22.csv', index = False) 

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X, y = sm.fit_resample(X,y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X,y)
knn_pred = knn.predict(test3)


cat_sub = sample_submission.copy()
cat_sub.subscribed = knn_pred
cat_sub.to_csv('23.csv', index = False)

In [ ]:
from catboost import CatBoostClassifier
cat_model=CatBoostClassifier(max_depth=3)
cat_model.fit(X,y, verbose=400, early_stopping_rounds=350)
#when I set the max_depth to 3, my accuracy increased tofrom 0.917 to 0.92
cat_pred= cat_model.predict(test3)
cat_sub = sample_submission.copy()
cat_sub.subscribed = cat_pred
cat_sub.to_csv('24.csv', index = False)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.education.unique()

In [ ]:
df_train['education'] = df_train['education'].replace('basic.9y', 'basic')
df_train['education'] = df_train['education'].replace('basic.6y', 'basic')
df_train['education'] = df_train['education'].replace('basic.4y', 'basic')

In [ ]:
df_test['education'] = df_test['education'].replace('basic.9y', 'basic')
df_test['education'] = df_test['education'].replace('basic.6y', 'basic')
df_test['education'] = df_test['education'].replace('basic.4y', 'basic')

In [ ]:
df_train.education.unique()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()

In [ ]:
col = ['customer_id', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
for cols in col:
    df_train[cols] = lb.fit_transform(df_train[cols])
    

In [ ]:
col = ['customer_id', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
for cols in col:
    df_test[cols] = lb.fit_transform(df_test[cols])
    

In [ ]:
# df_train.info() , df_test.info()

In [ ]:
#picking X and y
X = df_train.drop(["customer_id", "subscribed"], axis =1)
y = df_train["subscribed"]

X_test = df_test.drop(["customer_id"], axis = 1)


In [ ]:

from sklearn.model_selection import train_test_split  #change test size from .20 to .28

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, stratify = y, random_state = 42)

X_train.shape, X_val.shape

In [ ]:
X.info()

In [ ]:
filter = VarianceThreshold(0.5)
X_train = filter.fit_transform(X_train)
X_val = filter.transform(X_val)
X_test = filter.transform(X_test)

X_train.shape, X_val.shape, X_test.shape

In [ ]:
scaler = StandardScaler()   # MinMaxScaler
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
import random
seed = 1994
random.seed(seed)
np.random.seed(seed)

In [ ]:
def crossVal(model):
    avg = 0
    for fold, (tr, te) in enumerate(skfolds.split(xtrainval, ytrainval), 1):
        train_x, test_x = xtrainval.iloc[tr], xtrainval.iloc[te]
        train_y, test_y = ytrainval.iloc[tr], ytrainval.iloc[te]
        
        model.fit(train_x, train_y, eval_set=[(train_x, train_y), (test_x, test_y)], 
                  early_stopping_rounds=200, verbose=False)
        preds = model.predict_proba(test_x)[:,1]
        logloss = log_loss(test_y, preds)
        print(f'Fold {fold}: Logloss: {logloss}')
        avg += logloss
    free = gc.collect()
    print(f'Avg Logloss: {avg/fold}')

In [ ]:
def crossValPred(model):
    avg = 0
    test_scores = []
    for fold, (tr, te) in enumerate(skfolds.split(X, y), 1):
        train_x, test_x = X.iloc[tr], X.iloc[te]
        train_y, test_y = y.iloc[tr], y.iloc[te]
        
        model.fit(train_x, train_y, eval_set=[(train_x, train_y), (test_x, test_y)], 
                  early_stopping_rounds=200, verbose=False)
        
        preds = model.predict_proba(test_x)[:, 1]
        logloss = log_loss(test_y, preds)
        print(f'Fold {fold}: Logloss: {logloss}')
        avg += logloss
        
        
        test_pred = model.predict_proba(test_c)[:,1]
        test_scores.append(test_pred)
    print(f'Avg Logloss: {avg/fold}')
    free = gc.collect()
    return np.mean(test_scores, 0)

In [ ]:
xtrainval, xtest, ytrainval, ytest = train_test_split(X, y, test_size=0.18, stratify=y, random_state=seed)
xtrain, xval, ytrain, yval = train_test_split(xtrainval, ytrainval, test_size=0.20, stratify=ytrainval, random_state=seed)

In [ ]:
xtrain.shape, xval.shape, xtest.shape

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, plot_confusion_matrix, plot_roc_curve
print("done")

In [ ]:
import catboost as cat
catm = cat.CatBoostClassifier(random_seed=seed, bootstrap_type='Bernoulli', 
                              iterations=1000, silent=True, eval_metric='Logloss')

In [ ]:

skfolds= StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)

In [ ]:
xtrainval, xtest, ytrainval, ytest = train_test_split(X, y, test_size=0.18, stratify=y, random_state=seed)
xtrain, xval, ytrain, yval = train_test_split(xtrainval, ytrainval, test_size=0.20, stratify=ytrainval, random_state=seed)

In [ ]:
# catm = cat.CatBoostClassifier(random_seed=seed, bootstrap_type='Bernoulli', 
#                               iterations=1000, silent=True, eval_metric='Logloss')

In [ ]:
%%time
catm.fit(X, y, eval_set=[(X,y)], early_stopping_rounds=350, verbose=200); #stopping from 350 to 300

In [ ]:
X.columns

In [ ]:
X_test.reshape(1,-1)

In [ ]:
cat_pred = catm.predict(X_test)

In [ ]:
import os, gc, warnings
warnings.filterwarnings('ignore')
gc.collect()
crossVal(catm)

In [ ]:
test_c = df_test.drop(columns='customer_id')

In [ ]:
cat_pred = crossValPred(catm)

In [ ]:
xtrainval, xtest, ytrainval, ytest = train_test_split(X, y, test_size=0.18, stratify=y, random_state=seed)
xtrain, xval, ytrain, yval = train_test_split(xtrainval, ytrainval, test_size=0.28, stratify=ytrainval, random_state=seed)

In [ ]:

lgbm = lgb.LGBMClassifier(random_state=seed, n_estimators=1000, metric='binary_logloss', learning_rate=0.04, subsample=0.7,
                         colsample_bytree=0.8, num_leaves=64, reg_alpha=4, min_child_samples=5,subsample_freq=2)

In [ ]:
lgbm.fit(xtrain, ytrain, eval_set=[(xtrain, ytrain), (xval,yval)], verbose=200, early_stopping_rounds=200);

In [ ]:
crossVal(lgbm)

In [ ]:
lgb_pred = crossValPred(lgbm)

In [ ]:
blend = (0.6 * lgb_pred + 0.4)

In [ ]:
sample_submission = pd.read_csv("sample_submission.csv")

In [ ]:
test_id = sample_submission.customer_id

In [ ]:
lgb_df =  pd.DataFrame({'customer_id' : test_id, 'subscribed' : lgb_pred})
cat_df =  pd.DataFrame({'customer_id' : test_id, 'subscribed' : cat_pred})
blend_df =  pd.DataFrame({'customer_id' : test_id, 'subscribed' : blend})

In [ ]:

lgb_df.to_csv('lgblast.csv', index=False)
cat_df.to_csv('catlast.csv', index=False)
blend_df.to_csv('blendlast.csv', index=False)